In [99]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [100]:
available_functions = {}

In [101]:
avalible_tools = [{"type": "retrieval"}]

In [126]:
# A Class to Manage All Open API Assistant Calls and Functions
from openai.types.beta.threads import Run, ThreadMessage
from openai.types.beta.thread import Thread
from openai.types.beta.assistant_create_params import Tool
from openai.types.beta.assistant import Assistant

from typing import Literal


import time
import json

class PDFChatManager:
    def __init__(self, model: str = "gpt-3.5-turbo-1106"):
        self.client = OpenAI()
        self.model = model
        self.assistant = None
        self.thread = None
        self.run = None

    def list_files(self, purpose: str = 'assistants') -> list:
        """Retrieve a list of files with the specified purpose."""
        files = client.files.list(purpose=purpose)
        file_list = files.model_dump()
        return file_list['data'] if 'data' in file_list else []

    def find_file_id_by_name(self, filename: str, purpose: str = 'assistants') -> str:
        """Check if the file exists in the OpenAI account and return its ID."""
        files = self.list_files(purpose=purpose)
        for file in files:
            print("file['filename'] == filename", file['filename'] == filename)
            if file['filename'] == filename:
                print("file['id']", file['id'])
                return file['id']
        return None

    def create_file(self, file_path: str, purpose: str='assistants') -> str:
        """Create or find a file in OpenAI. 
        https://platform.openai.com/docs/api-reference/files/list
        If file is already uploaded with same name then 
        we will use it rather than creating a new one. """
        
        existing_file_id = self.find_file_id_by_name(file_path, purpose)

        print("found existing file...", existing_file_id)

        if existing_file_id:
            self.file_id = existing_file_id
            return existing_file_id
        else:
            with open(file_path, "rb") as file:
                file_obj = self.client.files.create(file=file, purpose=purpose)
                self.file_id = file_obj.id
                return file_obj.id

    def list_assistants(self) -> list:
        """Retrieve a list of assistants."""
        assistants_list = self.client.beta.assistants.list()
        assistants = assistants_list.model_dump()
        return assistants['data'] if 'data' in assistants else []

    def modifyAssistant(self, assistant_id: str, new_instructions: str, tools: list[Tool], file_obj: list[str]) -> Assistant:
        """Update an existing assistant."""
        print("Updating edisting assistant...")
        self.assistant = self.client.beta.assistants.update(
            assistant_id=assistant_id,
            instructions=new_instructions,
            tools=tools,
            model=self.model,
            file_ids=file_obj
        )
        return self.assistant

    def find_and_set_assistant_by_name(self, name: str, instructions: str, tools: list[Tool], file_obj: list[str]) -> None:
        """Find an assistant by name and set it if found."""
        assistants = self.list_assistants()
        print("Retrieved assistants list...")
        for assistant in assistants:
            if assistant['name'] == name:
                print("Found assistant...",  assistant['name'] == name)
                print("Existing Assitant ID", assistant['id'])
                # self.assistant = assistant
                self.modifyAssistant( 
                    assistant_id=assistant['id'],
                    new_instructions=instructions,
                    tools=tools,
                    file_obj=file_obj
                    )
                break

    def create_assistant(self, name: str, instructions: str, tools: list[Tool], file_obj: list[str]) -> None:
        """Create or find an assistant."""
        self.find_and_set_assistant_by_name(  
                name=name,
                instructions=instructions,
                tools=tools,
                file_obj=file_obj)
        if not self.assistant:
            print("Creating new assistant...")
            self.assistant = self.client.beta.assistants.create(
                name=name,
                instructions=instructions,
                tools=tools,
                model=self.model,
                file_ids=file_obj
            )

    def create_thread(self) -> Thread:
        self.thread = self.client.beta.threads.create()
        return self.thread

    def add_message_to_thread(self, role: str, content: str) -> None:
        self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role=role,
            content=content
        )

    def run_assistant(self, instructions: str) -> Run:
        self.run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
            instructions=instructions
        )
        return self.run

    def wait_for_completion(self, run: Run, thread: Thread) -> Run:

        while run.status in ["in_progress", "queued"]:
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=self.run.id
            )
            print(f"Run is {run.status} ...")
            time.sleep(3)  # Wait for 3 seconds before checking again

            if run_status.status == 'completed':
                processed_response = self.process_messages()
                return processed_response
                # break
            elif run_status.status == 'requires_action':
                print("Function Calling ...")
                self.call_required_functions(run_status.required_action.submit_tool_outputs.model_dump())
            elif run.status == "failed":
                print("Run failed.")
                break
            else:
                print(f"Waiting for the Assistant to process...: {run.status}")

    def process_messages(self) -> list[ThreadMessage]:
        messages: list[ThreadMessage] = self.client.beta.threads.messages.list(thread_id=self.thread.id)
        return messages

    def call_required_functions(self, required_actions: dict):
        tool_outputs = []

        for action in required_actions["tool_calls"]:
            function_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            print('function_name', function_name)
            print('function_arguments', arguments)

            if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    output = function_to_call(**arguments)
                    tool_outputs.append({
                        "tool_call_id": action['id'],
                        "output": output,
                    })

            else:
                raise ValueError(f"Unknown function: {function_name}")

        print("Submitting outputs back to the Assistant...")
        self.client.beta.threads.runs.submit_tool_outputs(
            thread_id=self.thread.id,
            run_id=self.run.id,
            tool_outputs=tool_outputs
        )


    # List, Modify And Destroy Files & Assistants
    def deleteFile(self) -> dict[str, Literal["id", "object", "deleted"]]:
        return self.client.files.delete(self.file_id)
    
    def retriveAssistantFile(self) -> dict[str, Literal["id", "object", "created_at", "assistant_id"]]:
        return client.beta.assistants.files.retrieve(
            assistant_id=self.assistant.id,
            file_id=self.file_id
        )
    
    def delAssistantFile(self) -> dict[str, Literal["id", "object", "deleted"]]:
        return client.beta.assistants.files.delete(
            assistant_id=self.assistant.id,
            file_id=self.file_id
        )
    
    def deleteAssistant(self) -> dict[str, Literal["id", "object", "deleted"]]:
        return self.client.beta.assistants.delete(self.assistant.id)




In [127]:
# Show Messages and Plot Images in Financial Analysis If ANY

import requests
from IPython.display import Image, display


def download_and_save_image(file_id: str, save_path: str) -> None:
    """
    Downloads an image from OpenAI using its file ID and saves it to the specified path.

    Args:
    - file_id (str): The ID of the file to download.
    - save_path (str): The path where the image will be saved.

    Returns:
    - None
    """
    # Construct the URL to download the image
    download_url = f"https://api.openai.com/v1/files/{file_id}/content"

    # Perform the HTTP GET request to download the image
    response = requests.get(download_url, headers={"Authorization": f"Bearer {os.environ.get("OPENAI_API_KEY")}"})

    # Check if the request was successful
    if response.status_code == 200:
        # Write the image to the specified file
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image downloaded and saved to {save_path}")
    else:
        print(f"Failed to download image: HTTP Status Code {response.status_code}")


def pretty_print(messages: list[ThreadMessage]) -> None:
    print("# Messages")
    for message in messages.data:
        role_label = "User" if message.role == "user" else "Assistant"
        # Check the type of message content and handle accordingly
        for content in message.content:
            if content.type == "text":
                message_content = content.text.value
                print(f"{role_label}: {message_content}\n")
                print()
            elif content.type == "image_file":
                # Handle image file content, e.g., print the file ID or download the image
                image_file_id = content.image_file.file_id
                # Define a path to save the image
                image_save_path = f"image_{image_file_id}.png"
                # Download and save the image
                print(f"{role_label}: Image file ID: {image_file_id}")
                download_and_save_image(image_file_id, image_save_path)

                # Display the image within Jupyter Notebook
                display(Image(filename=image_save_path))

In [137]:
def chat_with_pdf(file_path: str, assistant_intructions: str, question_to_ask: str) -> (ThreadMessage, PDFChatManager, Thread, str, Assistant):
    pdf_assistant_manager: PDFChatManager = PDFChatManager()

    # 00 Create a file
    file_obj : str = pdf_assistant_manager.create_file(file_path=file_path)

    # 01 Create an assistant
    pdf_assistant = pdf_assistant_manager.create_assistant(name="New PDF Assistant", instructions=assistant_intructions, tools=avalible_tools, file_obj=[file_obj])

    # 02 Create a thread
    thread = pdf_assistant_manager.create_thread()

    # 03 Add a message to the thread
    pdf_assistant_manager.add_message_to_thread(role="user", content=question_to_ask)

    # 04 Run the assistant
    run = pdf_assistant_manager.run_assistant(instructions="")

    # 05 Wait for the assistant to complete
    messages = pdf_assistant_manager.wait_for_completion(run=run, thread=pdf_assistant_manager.thread)

    #06 return response to be displayed
    return messages, pdf_assistant_manager, run, thread, file_obj, pdf_assistant

    

In [133]:
ASSISTANT_SEED_PROMPT =  """ 

You are a specialized AI Assistant who efficiently manages and extracts information from PDF documents. 

Your role is to assist a diverse range of users, from business professionals to individuals, in navigating and understanding their PDF files. When interacting with users:

1. Identify the User's Objective from their Query

2. Request Specific Details: Encourage users to be specific about their needs. For instance, if they want to extract data, ask them to define the type of data (like dates, names, financial figures).

3. Understand the Context: Inquire about the nature of the document (e.g., financial report, academic paper) to tailor your assistance accordingly.

4. Communicate Clearly: Use straightforward, easy-to-understand language in your responses. Avoid technical jargon unless the user is comfortable with it.

5. Logical Question Sequencing: If a task requires multiple steps, guide the user through them in a logical order. For example, start with general extraction before moving to specific data points.

6. Prepare for Diverse Responses: Be ready to handle a range of user queries and rephrase your questions or guidance based on user feedback.

7. Iterate Based on User Feedback: If the user's response indicates misunderstanding, rephrase your guidance or provide additional clarifications.

Remember, your goal is to make the user's interaction with their PDFs more efficient and productive, respecting their privacy and time constraints. Don't say I don't know, instead, return you understanding or ask for more information.

"""

In [138]:
# Using Assistant
messages, pdf_assistant_manager, run, thread, file_obj, pdf_assistant = chat_with_pdf(file_path="the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf.pdf", assistant_intructions=ASSISTANT_SEED_PROMPT, question_to_ask="Share 3 Startup Ideas from the niches discussed in CH 2")

file['filename'] == filename True
file['id'] file-56czFLm3VqrD1ak1WcwXA2Uh
found existing file... file-56czFLm3VqrD1ak1WcwXA2Uh
Retrieved assistants list...
Found assistant... True
Existing Assitant ID asst_wPhZ5gjkToeVzCVLeQCxcWex
Updating edisting assistant...
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...


In [139]:
pretty_print(messages)

# Messages
Assistant: From Chapter 2 of the document "The Economic Potential of Generative AI: The Next Productivity Frontier," we can extract three potential startup ideas in different niches.

1. **Software Engineering Transformation:** One startup idea can focus on leveraging generative AI to revolutionize software engineering processes. This could involve developing AI tools to assist software engineers in various tasks such as analyzing, cleaning, and labeling large volumes of data; creating multiple IT architecture designs and iterating on potential configurations; coding with AI assistance to reduce development time; enhancing functional and performance testing; using AI insights to diagnose issues, suggest fixes, and predict high-priority areas for improvement【13†source】.

2. **Product R&D Transformation:** Another startup idea could be centered around using generative AI to transform product research and development processes. This could include using generative AI to enhance 

In [63]:
pdf_assistant_manager.add_message_to_thread(role="user", content="Share 3 Startup Ideas from the niches discussed in CH 2")

In [66]:
# 04 Run the assistant
pdf_assistant_manager.run_assistant(instructions="")

 # 05 Wait for the assistant to complete
messages = pdf_assistant_manager.wait_for_completion(run=run, thread=pdf_assistant_manager.thread)

Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...
Waiting for the Assistant to process...: queued
Run is queued ...


In [67]:
pretty_print(messages)

# Messages
Assistant: I have found some relevant information in the document regarding the potential use of Generative AI in marketing and sales, software engineering, and product R&D. However, I couldn't locate specific startup ideas in Chapter 2. Here are some key points from the document:

1. Marketing and Sales Transformation:
   - Generative AI can be used to create personalized messages tailored to individual customer interests, preferences, and behaviors. It can also facilitate efficient and effective content creation, enhance use of data, optimize SEO, and personalize product discovery and search【11†source】.

2. Software Engineering Transformation:
   - Generative AI can assist in analyzing, cleaning, labeling large volumes of data, creating IT architecture designs, coding, testing, and maintenance. It can significantly reduce the time spent on certain activities and accelerate the coding process【11†source】.

3. Product R&D Transformation:
   - Generative AI can enhance early r

In [64]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [68]:
show_json(messages)

{'data': [{'id': 'msg_7ysdNsg7iUnMo8Xd32uaqSpz',
   'assistant_id': 'asst_E4ur92p0IBKCtyHdvhFfU6fr',
   'content': [{'text': {'annotations': [],
      'value': "I have found some relevant information in the document regarding the potential use of Generative AI in marketing and sales, software engineering, and product R&D. However, I couldn't locate specific startup ideas in Chapter 2. Here are some key points from the document:\n\n1. Marketing and Sales Transformation:\n   - Generative AI can be used to create personalized messages tailored to individual customer interests, preferences, and behaviors. It can also facilitate efficient and effective content creation, enhance use of data, optimize SEO, and personalize product discovery and search【11†source】.\n\n2. Software Engineering Transformation:\n   - Generative AI can assist in analyzing, cleaning, labeling large volumes of data, creating IT architecture designs, coding, testing, and maintenance. It can significantly reduce the time